# scores

> Get scores (z/b/npi/%) for each well

In [ ]:
#| default_exp scores

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import Union, List, Tuple

import numpy as np
import pandas as pd

from hitseekpy.proc import Plate

In [ ]:
# load plate data and make sure wells are properly labeled
df = pd.read_csv("assay.1156.0009.tsv", delimiter="\t")
df[["WELL", "RAW_VALUE_A"]]

,WELL,RAW_VALUE_A
0,A01,1960.0
1,A02,1400.0
2,A03,1600.0
3,A04,1560.0
4,A05,1640.0
...,...,...
379,P20,1800.0
380,P21,1240.0
381,P22,1520.0
382,P23,1200.0


In [ ]:
# extract well values into a 2d array
p = Plate(df, n_rows=16, n_cols=24, well_column="WELL", value_column="RAW_VALUE_A")

# check that they match the df
print(p.vals[0, :5])
print(p.vals[-1, -5:])

[1960. 1400. 1600. 1560. 1640.]
[1800. 1240. 1520. 1200. 1480.]


In [ ]:
#| export
def _plate_idx_to_flat(index_list: List[Tuple[int, int]], to_shape: Tuple[int, int]):
    multi_index_np = np.array(list(zip(*index_list)))
    return np.ravel_multi_index(multi_index_np, to_shape)

def get_percentages_of_controls(vals: np.array, control_idx: List[Tuple[int, int]]):
    control_flat_idx = _plate_idx_to_flat(control_idx, vals.shape)
    control_vals = vals[control_flat_idx]
    return vals / np.mean(control_vals) * 100

def get_npis(vals: np.array, pos_control_idx: List[Tuple[int, int]], neg_control_idx: List[Tuple[int, int]]):
    pos_control_flat_idx = _plate_idx_to_flat(pos_control_idx, vals.shape)
    neg_control_flat_idx = _plate_idx_to_flat(neg_control_idx, vals.shape)
    pos_mean = np.mean(vals[pos_control_flat_idx])
    neg_mean = np.mean(vals[neg_control_flat_idx])
    return (pos_mean - vals) / (pos_mean - neg_mean)

def get_z_scores(vals: np.array, robust: bool = True):
    mu = np.median(vals) if robust else np.mean(vals)
    sigma = np.mean(np.abs(vals - mu)) if robust else np.std(vals)
    return (vals - mu) / sigma

def get_b_scores(vals: np.array, iters: int = 5):
    # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5001608/pdf/gkw554.pdf
    # https://mgimond.github.io/ES218/Week11a.html
    # https://github.com/Palpatineli/median_polish/blob/master/median_polish/main.py
    centered = vals.copy()
    centered -= np.median(centered)
    median_margins = [0] * 2
    margins = [np.zeros(shape=centered.shape[idx]) for idx in range(2)]
    dim_mask = np.ones(2, dtype=int)

    for _ in range(iters):
        for dim_id in range(2):
            rest_dim = 1 - dim_id
            temp_median = np.median(centered, axis=rest_dim)
            margins[dim_id] += temp_median
            median_margins[rest_dim] = np.median(margins[rest_dim])
            margins[rest_dim] -= median_margins[rest_dim]
            dim_mask[dim_id] = -1
            centered -= temp_median.reshape(dim_mask)
            dim_mask[dim_id] = 1

    mad = np.median(np.abs(vals - np.median(vals)))

    return {
        "normalized": centered / mad,
        "row_effect": margins[1],
        "column_effect": margins[0]
    }

def get_ssmd(pop_1, pop_2):
    return (np.mean(pop_1) - np.mean(pop_2)) / np.sqrt(np.var(pop_1) + np.var(pop_2))

In [ ]:
get_b_scores(p.vals)

{'normalized': array([[ 2.26025391e+00, -8.49609375e-01,  8.83789062e-01,
          5.75927734e-01,  1.80175781e-01,  1.71679688e+00,
          1.63574219e-02,  1.56030273e+00,  2.36328125e-01,
         -9.89746094e-01, -1.36718750e-02, -9.62158203e-01,
         -5.50781250e-01,  1.08422852e+00, -5.41503906e-01,
         -1.88940430e+00, -1.29956055e+00,  1.52124023e+00,
         -3.34228516e-01, -9.61669922e-01,  1.90625000e+00,
          1.65600586e+00, -1.42382812e+00, -1.67382812e+00],
        [-3.15429688e-01,  2.32470703e+00,  5.58105469e-01,
          2.44140625e-04,  3.54492188e-01, -1.35888672e+00,
         -5.59326172e-01,  2.98461914e+00,  1.60644531e-01,
          1.68457031e+00, -8.39355469e-01,  3.46215820e+00,
         -3.76464844e-01,  5.08544922e-01, -3.67187500e-01,
          7.84912109e-01, -6.25244141e-01, -3.04443359e-01,
         -2.40991211e+00, -3.73535156e-02, -6.69433594e-01,
          8.03222656e-02,  2.75048828e+00,  4.88281250e-04],
        [ 7.27539062e-01

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()